In [35]:
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
import re
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

In [36]:
pd.set_option('display.max_rows', 100)  # Show up to 100 rows
pd.set_option('display.max_columns', 20)  # Show up to 20 columns
pd.set_option('display.max_colwidth', None)

In [37]:
# Define hierarchical genre mapping
genre_mapping = {
    'Romance': ['romance', 'love', 'romantic comedy', 'romantic suspense', 'historical romance', 'romance: contemporary', 'romance: historical', 'romance: regency', 'erotica', 'fiction romance contemporary', 'fiction erotica general', 'love stories', 'erotic fiction', 'fiction romance general', 'dark fantasy'],
    'Thriller': ['thriller', 'suspense', 'mystery', 'crime', 'detective', 'psychological thriller', 'thrillers & suspense', 'fiction thrillers general', 'fiction thrillers suspense', 'action and adventure fiction'],
    'Fantasy': ['fantasy', 'magic', 'witches', 'wizards', 'epic fantasy', 'urban fantasy', 'westerns', 'american drama', 'fantasy fiction', 'fiction fantasy general', 'imaginary places', 'juvenile fantasy fiction'],
    'Science Fiction': ['science fiction', 'sci-fi', 'space opera', 'dystopian', 'cyberpunk', 'imaginary places', 'good and evil', 'time travel', 'alternative history'],
    'Historical Fiction': ['historical fiction', 'historical', 'world war ii', 'historical mystery', 'alternative history'],
    'Young Adult': ['young adult', 'teen', 'coming of age', 'ya fiction', 'ya romance', 'teen & young adult', 'young adult fiction', 'teenage girls', 'dark fantasy', 'homosexuality', 'coming of age', 'good and evil', 'friendship'],
    'Horror': ['horror', 'supernatural', 'vampires', 'zombies', 'ghosts', 'post apocalyptic', 'gothic', 'horror fiction', 'horror tales'],
    'Mystery': ['mystery', 'detective', 'crime', 'cozy mystery', 'whodunit', 'women sleuths', 'detective stories', 'mystery and detective stories', 'fiction mystery & detective traditional', 'crime novel'],
    'Adventure': ['adventure', 'action', 'quest', 'treasure hunt', 'exploration', 'action & adventure', 'action and adventure fiction', 'action & adventure'],
    'Literary Fiction': ['literary fiction', 'contemporary fiction', 'realistic fiction', 'literature & fiction', 'american drama', 'romans, nouvelles', 'epic', 'epic literature', 'novel', 'hindu mythology', 'homosexuality'],
    'Humor': ['humor', 'comedy', 'satire', 'funny', 'parody'],
    'Biography': ['biography', 'autobiography', 'memoir', 'personal memoir', 'life story'],
    'Classics': ['classics', 'literature - classics / criticism'],
    'Children': ['reading level grade 9', 'reading level grade 8', 'reading level grade 11', 'reading level grade 10', 'reading level grade 12', 'comics', 'comic books, strips', 'friendship', 'juvenile fantasy fiction', 'juvenile fiction'],
    'Other': ['non-classifiable']  # Default category for remaining genres
}

In [38]:
def preprocess_genre(genre):
    # Remove text within parentheses, brackets, and braces
    genre = re.sub(r"[\(\)\{\}\[\]]", "", genre)
    # Replace specific punctuation with space
    genre = re.sub(r"[:/\?-]", " ", genre)
    # Remove periods
    genre = re.sub(r"\.", "", genre)
    # Normalize whitespace to a single space
    genre = re.sub(r"\s+", " ", genre)
    # Convert to lowercase
    genre = genre.lower().strip()
    return genre

In [39]:
def match_genre_(genre_label, genre_mapping):
    matched_genres = []
    for category, genres in genre_mapping.items():
        for genre in genres:
            # Adjust the fuzzy matching threshold to be more inclusive
            if fuzz.token_sort_ratio(genre_label, genre) > 40:
                matched_genres.append(category)
                break  # Break out of inner loop if a match is found
    # If no match is found, assign the default category "Other"
    if not matched_genres:
        matched_genres.append('Other')
    return matched_genres

In [40]:
def match_genre__(genre_label, genre_mapping):
    matched_genres = []
    for category, genres in genre_mapping.items():
        # Check for substring matches
        for genre in genres:
            if genre in genre_label:
                matched_genres.append(category)
                break  # Break out of inner loop if a match is found
    
    # Remove duplicates from the matched genres list
    matched_genres = list(set(matched_genres))
    
    # If no match is found, assign the default category "Other"
    if not matched_genres:
        matched_genres.append('Other')
    
    return matched_genres


In [41]:
def match_genre(genre_label, genre_mapping):
    matched_genres = set()  # Use a set to avoid duplicate categories
    
    # Iterate over each key-value pair in genre_mapping
    for category, value in genre_mapping.items():
        # Check for substring matches in the genre_label for each value
        for genre in value:
            if genre in genre_label:
                # If a match is found, add the corresponding category to matched_genres
                matched_genres.add(category)
                break  # Break out of inner loop if a match is found
    
    # Iterate over each category and its corresponding list of genres in the genre_mapping dictionary
    for category, genres in genre_mapping.items():
        # Check for exact matches
        for genre in genres:
            if genre in genre_label:
                # If a match is found, add the corresponding category to matched_genres
                matched_genres.add(category)
                break  # Break out of inner loop if a match is found

    for category, genres in genre_mapping.items():
        for genre in genres:
            # Adjust the fuzzy matching threshold to be more inclusive
            if fuzz.token_sort_ratio(genre_label, genre) > 70:
                # If a match is found, add the corresponding category to matched_genres
                matched_genres.add(category)

    # If no match is found, assign the default category "Other"
    if not matched_genres:
        matched_genres.add('Other')
        
    return list(matched_genres)  # Convert set back to list for consistency

In [42]:
def clean_genres(genres):
    # Apply preprocess_genre to each item after splitting
    return [preprocess_genre(genre) for genre in genres.split('||') if preprocess_genre(genre)]

In [43]:
user = 'root'
password = 'nats'
host = 'localhost'
port = '3306' 
database = 'BookMate'

In [44]:
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')

In [50]:
query = 'SELECT * FROM book LIMIT 500000'
df_books = pd.read_sql(query, engine)

In [ ]:
df_books['clean_genre'] = df_books['Genres'].apply(clean_genres)

In [47]:
df_books['broad_genre'] = df_books['clean_genre'].apply(match_genre, args=(genre_mapping,))

In [48]:
df_books

,Title,ISBN,Genres,clean_genre,broad_genre
0,Orcas Island mystery,,|Brothers and sisters -- Fiction.||Twins -- Fiction.||Vacations -- Fiction.||Seventh-Day Adventists -- Fiction.||Mystery and detective stories.||Orcas Island (Wash.) -- Fiction.|\r,"[|brothers and sisters fiction, twins fiction, vacations fiction, seventh day adventists fiction, mystery and detective stories, orcas island wash fiction|]",[Mystery]
1,Say you love me,,|Large type books||England -- Social life and customs -- 19th century -- Fiction|\r,"[|large type books, england social life and customs 19th century fiction|]",[Other]
2,Matilda the moocher,,|Borrowing and lending -- Fiction.||Neighbors -- Fiction.||Friendship -- Fiction.||Schools -- Fiction.|\r,"[|borrowing and lending fiction, neighbors fiction, friendship fiction, schools fiction|]",[Other]
3,World Studies,9780131669697,|People & Places - Africa||Juvenile Nonfiction||Children: Young Adult (Gr. 7-9)|\r,"[|people & places africa, juvenile nonfiction, children young adult gr 7 9|]",[Other]
4,Puffin Bk of Stories for 7 Yr-Olds,9780140374605,|Short stories||General||Juvenile Fiction||Children: Grades 2-3|\r,"[|short stories, general, juvenile fiction, children grades 2 3|]",[Children]
...,...,...,...,...,...
9995,Lizzie and the Guernsey Gang,9781945831133,"Fiction, historical, general\r","[fiction, historical, general]","[Thriller, Romance, Literary Fiction, Historical Fiction]"
9996,Seasons of Lesbian Erotica,9781504504317,"Fiction, erotica, general\r","[fiction, erotica, general]","[Fantasy, Romance]"
9997,Killer Debt,9781732236707,"|Fiction, historical, general||Fiction, mystery & detective, general|\r","[|fiction, historical, general, fiction, mystery & detective, general|]",[Other]
9998,Gabriel's Send-Off,9781504504607,"Fiction, erotica, general\r","[fiction, erotica, general]","[Fantasy, Romance]"


In [49]:
genre_counts = df_books['broad_genre'].value_counts()
top_genres = genre_counts.head(50)
print(top_genres)

broad_genre
[Other]                                                                                   6826
[Children]                                                                                 463
[Thriller, Romance]                                                                        416
[Romance]                                                                                  337
[Literary Fiction, Thriller, Fantasy, Science Fiction, Romance, Young Adult, Children]     316
[Thriller]                                                                                 167
[Fantasy]                                                                                  156
[Fantasy, Romance]                                                                         131
[Thriller, Romance, Literary Fiction, Historical Fiction]                                  124
[Mystery]                                                                                  115
[Thriller, Adventure]                 